In [1]:
import os
import numpy as np
import pandas as pd
import json
import requests
import datetime as dt

from typing import Optional
from binance.client import Client
from time import sleep

In [2]:
BASE_SPOT_DATA_URL = "https://api.binance.com/api/v3/klines"
BASE_OPTION_INFO_URL = "https://eapi.binance.com/eapi/v1/exchangeInfo"

In [4]:
json.loads(requests.get(BASE_OPTION_INFO_URL).text)

{'timezone': 'UTC',
 'serverTime': 1692543177495,
 'optionContracts': [{'id': 1,
   'baseAsset': 'SOL',
   'quoteAsset': 'USDT',
   'underlying': 'SOLUSDT',
   'settleAsset': 'USDT'},
  {'id': 2,
   'baseAsset': 'BTC',
   'quoteAsset': 'USDT',
   'underlying': 'BTCUSDT',
   'settleAsset': 'USDT'},
  {'id': 3,
   'baseAsset': 'ETH',
   'quoteAsset': 'USDT',
   'underlying': 'ETHUSDT',
   'settleAsset': 'USDT'},
  {'id': 4,
   'baseAsset': 'BNB',
   'quoteAsset': 'USDT',
   'underlying': 'BNBUSDT',
   'settleAsset': 'USDT'},
  {'id': 5,
   'baseAsset': 'XRP',
   'quoteAsset': 'USDT',
   'underlying': 'XRPUSDT',
   'settleAsset': 'USDT'},
  {'id': 6,
   'baseAsset': 'DOGE',
   'quoteAsset': 'USDT',
   'underlying': 'DOGEUSDT',
   'settleAsset': 'USDT'}],
 'optionAssets': [{'id': 1, 'name': 'USDT'}],
 'optionSymbols': [{'contractId': 3,
   'expiryDate': 1695974400000,
   'filters': [{'filterType': 'PRICE_FILTER',
     'minPrice': '771.5',
     'maxPrice': '972.1',
     'tickSize': '0.1'},


In [ ]:

# get kline data given params
data = pd.DataFrame(json.loads(requests.get(url, params= params).text))
data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
data.index   = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
extract_item  = [ 'open', 'high', 'low', 'close', 'volume', 'num_trades', 'taker_base_vol']


In [17]:
# set up Binance Client
api_key    = os.environ.get('binance_api_key')
api_secret = os.environ.get('binance_secret_key')

client = Client(api_key, api_secret)

In [33]:
client.get_server_time() # UNIX format??

{'serverTime': 1692448731314}

# Spot Data Fetching

In [142]:
BASE_SPOT_DATA_URL = 'https://api.binance.com/api/v3/klines'
FULL_DAY = 24 * 60 * 60
HALF_DAY = 12 * 60 * 60
MINUTE_1 = 60

def string_to_timestamp(date: str) -> dt.datetime:
    """_summary_

    Args:
        date (str): _description_

    Returns:
        dt.datetime: _description_
    """
    year, month, day = date.split('-')
    if month.startswith('0'):
        month = month[1]
    if day.startswith('0'):
        day = day[1]
    
    # prepare data as dt.datetime format
    year, month, day = int(year), int(month), int(day)
    return int(dt.datetime(year, month, day).timestamp())

def kline_data_from_api(
        params: dict, 
        url:str=BASE_SPOT_DATA_URL
    ) -> pd.DataFrame:
    """_summary_

    Args:
        params (dict): _description_
        url (_type_, optional): _description_. Defaults to 'https://api.binance.com/api/v3/klines'.

    Returns:
        pd.DataFrame: _description_
    """
    # get kline data given params
    data = pd.DataFrame(json.loads(requests.get(url, params= params).text))
    data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
    data.index   = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
    extract_item  = [ 'open', 'high', 'low', 'close', 'volume', 'num_trades', 'taker_base_vol']
    return data.loc[:,extract_item]

def fetch_kline_data(
        symbol: str, 
        interval: str, 
        start_date: str, 
        end_date: str,
        url: Optional[str]=None
    ) -> pd.DataFrame:
    """_summary_

    Args:
        symbol (str): _description_
        interval (str): _description_
        start_date (str): _description_
        end_date (str): _description_
        url (Optional[str], optional): _description_. Defaults to None.

    Returns:
        pd.DataFrame: _description_
    """
    
    if url is None:
        url = BASE_SPOT_DATA_URL
        
    start_date = string_to_timestamp(start_date)
    end_date   = string_to_timestamp(end_date) + FULL_DAY # to get data up to 23:59:00
    cut_point  = start_date + HALF_DAY
    
    agg_data = []
    while cut_point <= end_date:
        # fetch data based on the params dictionary.
        params   = {
                    'symbol': symbol, 
                    'interval': interval, 
                    'startTime': str(start_date * 1000), 
                    'endTime': str((cut_point - MINUTE_1) * 1000), 
                    'limit':1000
                }
        
        # append to the list.
        agg_data.append(kline_data_from_api(params, url))
        
        # shift the viewing window
        start_date = cut_point 
        cut_point  = start_date + HALF_DAY

    agg_data = pd.concat(agg_data, axis=0)
    agg_data = agg_data.drop_duplicates()
    return agg_data.astype(float)
    

In [143]:
# data fetching params
symbol   = 'XRPUSDT'
interval = '1m'
start    = '2022-01-01'
end      = '2023-08-15'

In [144]:
data = fetch_kline_data(symbol, interval, start, end)

In [163]:
start = dt.datetime.now()

In [164]:
end = dt.datetime.now()

In [169]:
time_taken = (end - start).total_seconds()

In [172]:
print(f'time taken: {time_taken//60} mins {round(time_taken%60,2)} seconds')

time taken: 0.0 mins 32.12 seconds


In [148]:
SPOT_DATA_PATH = '/Users/kibeom/Desktop/coding/coin_trading/data/spot/'

In [149]:
data.to_csv(SPOT_DATA_PATH + f"{symbol.lower()}_spot.csv")


In [151]:
dt.datetime.today().strftime('%Y-%m-%d')

'2023-08-19'

## websocket example

In [21]:
from time import sleep
from binance import ThreadedWebsocketManager